# Runtime Comparison - Blackjack

Compare the run time of different models: Q table, MLP, NDNF-MT and logic-based
programs.

Each model is run 100 episodes.

In [1]:
from datetime import datetime
from pathlib import Path
import sys

sys.path.append("..")

In [2]:
import gymnasium as gym
import numpy as np
import torch

In [3]:
from blackjack_common import get_target_policy, construct_model, blackjack_env_preprocess_obss
from eval.blackjack_ppo_rl_eval_common import eval_on_environments
from eval.blackjack_problog_rules_inference import blackjack_problog_context_gen_fn
from eval.problog_inference_common import prolog_inference_in_env_single_run

In [4]:
DEVICE = torch.device("cpu")
NUM_EPISODES = 10000

In [5]:
blackjack_env = gym.make("Blackjack-v1", render_mode=None)

# Q table

In [6]:
# Q table
target_policy = get_target_policy(Path("../results/BJ-TAB/BLACKJACK-TAB-snb/blackjack_tab_snb.csv"))

start_time = datetime.now().timestamp()

for _ in range(NUM_EPISODES):
    obs, _ = blackjack_env.reset()
    terminated, truncated = False, False
    while not terminated and not truncated:
        action = target_policy[obs]
        obs, _, terminated, truncated, _ = blackjack_env.step(action)

end_time = datetime.now().timestamp()

print(f"Time taken: {end_time - start_time}")
print(f"Avg time per episode: {(end_time - start_time) / NUM_EPISODES}")

Time taken: 0.40834999084472656
Avg time per episode: 4.083499908447266e-05


## MLP

In [7]:
# MLP
mlp_model = construct_model(
    num_latent=64,
    use_ndnf=False,
    use_decode_obs=True,
    use_eo=False,
    use_mt=True,
    share_layer_with_critic=False,
)
mlp_model.to(DEVICE)
sd = torch.load(
    "../blackjack_ppo_storage/blackjack_ppo_mlp_al64_3e5_3802/model.pth",
    map_location=DEVICE,
)
mlp_model.load_state_dict(sd)
mlp_model.eval()


/var/folders/64/b3z20xjs19938j8j9psfx4580000gn/T/ipykernel_94745/2575995003.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(


BlackjackMLPAgent(
  (actor): Sequential(
    (0): Linear(in_features=44, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=2, bias=True)
  )
  (critic): Sequential(
    (0): Linear(in_features=44, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [8]:
# Non-parallel

start_time = datetime.now().timestamp()

for _ in range(NUM_EPISODES):
    obs, _ = blackjack_env.reset()
    terminated, truncated = False, False
    while not terminated and not truncated:
        with torch.no_grad():
            obs_tensor = blackjack_env_preprocess_obss(
                obs_tuple=(
                    np.array([obs[0]]),
                    np.array([obs[1]]),
                    np.array([obs[2]]),
                ),
                use_ndnf=False,
                device=DEVICE,
            )
            action = mlp_model.get_actions(obs_tensor, use_argmax=False)[0]
        action = target_policy[obs]
        obs, _, terminated, truncated, _ = blackjack_env.step(action)

end_time = datetime.now().timestamp()
print(f"Time taken: {end_time - start_time}")
print(f"Avg time per episode: {(end_time - start_time) / NUM_EPISODES}")

Time taken: 2.0934181213378906
Avg time per episode: 0.00020934181213378905


In [9]:
# Parallel
start_time = datetime.now().timestamp()
eval_on_environments(
    model=mlp_model,
    device=DEVICE,
    use_argmax=False,
    eval_num_runs=NUM_EPISODES,
)
end_time = datetime.now().timestamp()

print(f"Time taken: {end_time - start_time}")
print(f"Avg time per episode: {(end_time - start_time) / NUM_EPISODES}")

Time taken: 1.213521957397461
Avg time per episode: 0.0001213521957397461


# NDNF-MT

In [10]:
# NDNF-MT
ndnf_mt_model = construct_model(
    num_latent=64,
    use_ndnf=True,
    use_decode_obs=True,
    use_eo=False,
    use_mt=True,
    share_layer_with_critic=False,
)
ndnf_mt_model.to(DEVICE)
sd = torch.load(
    "../blackjack_ppo_storage/blackjack_ppo_ndnf_mt_al64_3e5_aux10_7142/model_soft_2nd_mr_pruned.pth",
    map_location=DEVICE,
)
ndnf_mt_model.load_state_dict(sd)
ndnf_mt_model.eval()


/var/folders/64/b3z20xjs19938j8j9psfx4580000gn/T/ipykernel_94745/2648655595.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  sd = torch.load(


BlackjackNDNFMutexTanhAgent(
  (actor): NeuralDNFMutexTanh(
    (conjunctions): SemiSymbolic(in_features=44, out_features=64, layer_type=SemiSymbolicLayerType.CONJUNCTION,current_delta=1.00)
    (disjunctions): SemiSymbolicMutexTanh(in_features=64, out_features=2, layer_type=SemiSymbolicLayerType.DISJUNCTION,current_delta=1.00)
  )
  (critic): Sequential(
    (0): Linear(in_features=44, out_features=64, bias=True)
    (1): Tanh()
    (2): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [11]:
# Non-parallel

start_time = datetime.now().timestamp()

for _ in range(NUM_EPISODES):
    obs, _ = blackjack_env.reset()
    terminated, truncated = False, False
    while not terminated and not truncated:
        with torch.no_grad():
            obs_tensor = blackjack_env_preprocess_obss(
                obs_tuple=(
                    np.array([obs[0]]),
                    np.array([obs[1]]),
                    np.array([obs[2]]),
                ),
                use_ndnf=False,
                device=DEVICE,
            )
            action = ndnf_mt_model.get_actions(obs_tensor, use_argmax=False)[0][0]
        action = target_policy[obs]
        obs, _, terminated, truncated, _ = blackjack_env.step(action)

end_time = datetime.now().timestamp()
print(f"Time taken: {end_time - start_time}")
print(f"Avg time per episode: {(end_time - start_time) / NUM_EPISODES}")

Time taken: 8.037669897079468
Avg time per episode: 0.0008037669897079468


In [12]:
# Parallel
start_time = datetime.now().timestamp()
eval_on_environments(
    model=ndnf_mt_model,
    device=DEVICE,
    use_argmax=False,
    eval_num_runs=NUM_EPISODES,
)
end_time = datetime.now().timestamp()

print(f"Time taken: {end_time - start_time}")
print(f"Avg time per episode: {(end_time - start_time) / NUM_EPISODES}")

Time taken: 1.5832338333129883
Avg time per episode: 0.00015832338333129884


# ProbLog

In [13]:
# ProbLog

with open("../blackjack_ppo_storage/blackjack_ppo_ndnf_mt_al64_3e5_aux10_7142/problog_rules.pl", "r") as f:
    problog_rules = f.readlines()
problog_rules = [r.strip() for r in problog_rules]

In [14]:
start_time = datetime.now().timestamp()

for _ in range(10):
    prolog_inference_in_env_single_run(
        env=blackjack_env,
        problog_rules=problog_rules,
        num_actions=2,
        context_problog_gen_fn=blackjack_problog_context_gen_fn,
        use_argmax=False,
    )

end_time = datetime.now().timestamp()

print(f"Time taken: {end_time - start_time}")
print(f"Avg time per episode: {(end_time - start_time) / 10}")

Time taken: 11.855237007141113
Avg time per episode: 1.1855237007141113
